In [1]:
import sys
sys.path.append('..')

from misc import load, load_class_labels

dataset_root = "../dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train_info_cleaned.csv', 1, sample_rate)
test_info = load(dataset_root + 'labels/test.csv')
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values
test_image_names = test_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
test_labels = test_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [2]:
from cnn_classification import build_finetune_network, train_network
from tensorflow import keras
from keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input

save_path = "./tuned_models/ViTb16_noise_extended"

2024-01-07 12:52:20.077069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 12:52:20.077142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 12:52:20.105769: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 12:52:20.212484: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 12:52:21.222539: W tensorflow/compiler/tf2

In [ ]:
extension = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(251, activation='softmax')])

loss = 'categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy', keras.metrics.TopKCategoricalAccuracy(3)]

cut_layer = 'top_dropout'
non_trainable_cut_layer = 'block5e_add'

extended_model = build_finetune_network(EfficientNetV2B0, extension, cut_layer,
                                        optimizer, metrics, loss, non_trainable_cut_layer=non_trainable_cut_layer)

In [ ]:
train_loss, train_accuracy, val_loss, val_accuracy = train_network(extended_model,
                                                                   save_path,
                                                                   train_info,
                                                                   val_info,
                                                                   train_dir,
                                                                   epochs=10,
                                                                   augment=True,
                                                                   strong_augment=False,
                                                                   batch_size=64,
                                                                   preprocess_input=preprocess_input)

In [3]:
from tensorflow import keras
from cnn_classification import evaluate_model, predict
from vit_keras import vit

model = keras.models.load_model(save_path)
predictions, names = predict(model, val_dir, 64, preprocess_input=vit.preprocess_inputs)

/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2024-01-07 12:52:32.498760: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-07 12:52:32.652875: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA nod

1/1 [==============================] - 6s 6s/step


In [9]:
len(names)

11994

In [4]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

ordered_indices = np.argsort(names)
predictions = np.array(predictions)
ordered_names = np.argsort(test_image_names)

predictions_logit = np.argmax(predictions[ordered_indices], axis=1)
ordered_y = test_labels[ordered_names]


# cm = confusion_matrix(ordered_y, predictions_logit)
# disp = ConfusionMatrixDisplay(cm)
# disp.plot()
# plt.show()

In [16]:
predictions

array([[2.1613886e-07, 2.3317634e-06, 5.3297001e-05, ..., 7.3807205e-06,
        1.2932879e-05, 9.3355966e-06],
       [5.1270958e-06, 1.4018292e-04, 8.6029413e-06, ..., 3.4072767e-05,
        7.0335300e-05, 3.6837020e-05],
       [5.3782212e-05, 2.3184391e-04, 3.8748790e-06, ..., 1.3648253e-05,
        5.3050753e-04, 5.5727799e-04],
       ...,
       [9.6003464e-07, 6.9025127e-06, 2.8480813e-04, ..., 4.1882809e-05,
        7.3469197e-04, 2.2179386e-06],
       [3.5831906e-06, 2.1693970e-04, 3.8685359e-04, ..., 4.0897274e-05,
        5.0983366e-05, 2.8222898e-05],
       [1.6473721e-05, 1.7485270e-05, 1.1365205e-05, ..., 9.3104469e-04,
        1.2596528e-04, 6.7679401e-05]], dtype=float32)

In [5]:
from sklearn.metrics import classification_report

print(classification_report(ordered_y, predictions_logit, target_names=class_labels))

                               precision    recall  f1-score   support

                      macaron       0.82      0.89      0.85        55
                      beignet       0.70      0.90      0.79        61
                      cruller       0.73      0.70      0.71        53
                  cockle_food       0.76      0.57      0.65        51
                       samosa       0.62      0.83      0.71        41
                     tiramisu       0.61      0.65      0.63        52
                      tostada       0.73      0.53      0.61        57
                     moussaka       0.53      0.61      0.57        54
                     dumpling       0.60      0.61      0.61        49
                      sashimi       0.63      0.78      0.70        46
                        knish       0.67      0.50      0.57        48
                    croquette       0.48      0.51      0.49        47
                     couscous       0.27      0.87      0.41        45
     

/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [7]:
import os
import pandas as pd
os.makedirs("predictions", exist_ok=True)

pred_file = "predictions/ViT_cosine_clean"
np.save(pred_file, predictions)
df = pd.DataFrame({
    'filename': names
})
df.to_csv(pred_file + "_names.csv")

In [18]:
def top_k(k = 1):
    ohe = keras.utils.to_categorical(ordered_y, num_classes=251)
    m = keras.metrics.TopKCategoricalAccuracy(k)
    ordered_pred = predictions[ordered_indices]
    m.update_state(ohe, ordered_pred)
    return m.result().numpy()

In [103]:
top_k(1)

0.6135568

In [102]:
top_k(3)

0.7869768

In [22]:
from similarity_search.neural_similarity_search import extract_features
from feature_extraction import build_feature_extractor

builder = lambda : build_feature_extractor(EfficientNetV2B0, 'top_dropout')
extracted_features, filenames = extract_features(builder, train_dir, 100, keras.applications.efficientnet_v2.preprocess_input, 64, (224, 224))

100%|██████████| 1852/1852 [16:00<00:00,  1.93it/s] 


In [53]:
import pandas as pd

filenames = np.array(filenames)
filenames_df = pd.DataFrame({'filenames': filenames})
train_names_df = pd.DataFrame({'filenames': train_image_names})
filenames_df['filenames'].isin(train_names_df['filenames'])


# training_indices = np.isin(filenames, train_image_names)
training_indices = filenames_df.isin(train_names_df['filenames']).values.flatten()

training_features = extracted_features[training_indices]
training_filenames = filenames[training_indices]



In [77]:
np.save('./efficientnet_extracted_training.npy', extracted_features)
df = pd.DataFrame({
    'filenames': filenames
})
df.to_csv('./efficientnet_extracted_training_names.csv', header=None)

In [87]:
def find_similar(feature_extractor, query_image_path, database_features, database_names, preprocessing, norm=True,
                 output_number=10):
    from sklearn.preprocessing import normalize
    import cv2 as cv
    from scipy.spatial.distance import cdist
    if norm:
        database_features = normalize(database_features)
    query_image = cv.imread(query_image_path)[:, :, ::-1]
    query_image = cv.resize(query_image, (feature_extractor.input_shape[1], feature_extractor.input_shape[2]))
    query_image = preprocessing(query_image)
    img_array = np.zeros((1, feature_extractor.input_shape[1], feature_extractor.input_shape[2], 3))
    img_array[0] = query_image
    extracted_query = feature_extractor.predict(img_array)
    if norm:
        extracted_query = normalize(extracted_query)

    distances = cdist(extracted_query, database_features)[0]
    most_similar_indices = np.argsort(distances)[0:output_number]

    return database_names[most_similar_indices], distances[most_similar_indices]

i = 0
feature_extractor = build_feature_extractor(EfficientNetV2B0, 'top_dropout')
most_similar_filenames, distances = find_similar(feature_extractor, val_dir + names[i] , extracted_features, filenames, preprocess_input)
most_similar_filenames

1/1 [==============================] - 3s 3s/step


array(['train_060083.jpg', 'train_020225.jpg', 'train_055734.jpg',
       'train_015875.jpg', 'train_051474.jpg', 'train_062977.jpg',
       'train_112620.jpg', 'train_013079.jpg', 'train_051235.jpg',
       'train_100906.jpg'], dtype='<U16')

In [88]:
names[i]

'val_006589.jpg'

In [89]:
np.argmax(predictions[i])

130

In [92]:
gt = test_info[test_info['filename'] == names[i]]['label'].values[0]
train_info[train_info['label'] == gt]

,filename,label
13131,train_051219.jpg,43
13042,train_090428.jpg,43
13292,train_051556.jpg,43
13246,train_051172.jpg,43
13055,train_051566.jpg,43
...,...,...
13217,train_051428.jpg,43
13107,train_051608.jpg,43
13113,train_051549.jpg,43
13188,train_051392.jpg,43


In [95]:
from tqdm.notebook import tqdm_notebook
from sklearn.preprocessing import normalize
adjusted_predictions = []
pred_copy = np.zeros(predictions.shape)
normalized_features = normalize(extracted_features)
for i, p in tqdm_notebook(enumerate(predictions)):
    query_name = names[i]
    most_similar, _ = find_similar(feature_extractor, val_dir + query_name, normalized_features, filenames, preprocess_input, norm=False)
    available_classes = train_info[train_info['filename'].isin(most_similar)]['label'].unique()
    for c in available_classes:
        pred_copy[i, c] = p[c]

0it [00:00, ?it/s]

1/1 [==============================] - 0s 19ms/step


In [100]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

ordered_indices = np.argsort(names)
ordered_names = np.argsort(test_image_names)

predictions_logit = np.argmax(pred_copy[ordered_indices], axis=1)
ordered_y = test_labels[ordered_names]


# cm = confusion_matrix(ordered_y, predictions_logit)
# disp = ConfusionMatrixDisplay(cm)
# disp.plot()
# plt.show()

In [101]:
from sklearn.metrics import classification_report

print(classification_report(ordered_y, predictions_logit, target_names=class_labels))

                               precision    recall  f1-score   support

                      macaron       0.32      0.87      0.47        55
                      beignet       0.82      0.75      0.79        61
                      cruller       0.67      0.62      0.65        53
                  cockle_food       0.67      0.57      0.62        51
                       samosa       0.73      0.66      0.69        41
                     tiramisu       0.57      0.56      0.56        52
                      tostada       0.37      0.44      0.40        57
                     moussaka       0.52      0.41      0.46        54
                     dumpling       0.56      0.57      0.57        49
                      sashimi       0.46      0.57      0.51        46
                        knish       0.58      0.44      0.50        48
                    croquette       0.40      0.62      0.49        47
                     couscous       0.25      0.47      0.33        45
     

In [99]:
adjusted_predictions

[]